In [2]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import time
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

In [6]:
# reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols,encoding='latin-1')

# reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols,encoding='latin-1')

# reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('u.item', sep='|', names=i_cols,
encoding='latin-1')

# After loading the dataset, we should look at the content of each file (users, ratings, items).

# Looking at the user file
print("\nUser Data :")
print("shape : ", users.shape)
#print(users.head())

# We have 943 users in the dataset and each user has 5 features, i.e. user_ID, age, sex, occupation and zip_code. Now let’s look at the ratings file.

# Ratings Data
print("\nRatings Data :")
print("shape : ", ratings.shape)
#print(ratings.head())

# We have 100k ratings for different user and movie combinations. Now finally examine the items file.

# Item Data
print("\nItem Data :")
print("shape : ", items.shape)
#print(items.head())


User Data :
shape :  (943, 5)

Ratings Data :
shape :  (100000, 4)

Item Data :
shape :  (1682, 24)


In [ ]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [ ]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
#converting the unix timestamp of ratings to a  date
from datetime import datetime
def time_stamp(k):
  return datetime.fromtimestamp(k).strftime('%d-%m-%Y')
ratings['rating_date'] = ratings['unix_timestamp'].apply(time_stamp)
ratings.head()

,user_id,movie_id,rating,unix_timestamp,rating_date
0,196,242,3,881250949,04-12-1997
1,186,302,3,891717742,04-04-1998
2,22,377,1,878887116,07-11-1997
3,244,51,2,880606923,27-11-1997
4,166,346,1,886397596,02-02-1998


In [8]:
print(pd.DatetimeIndex(ratings['rating_date']).year.min())
print(pd.DatetimeIndex(ratings['rating_date']).year.max())

1997
1998


In [ ]:
#finding the no of days since the rating is given on movies from a specified date('01-11-1998')
date_format = "%d-%m-%Y"
def sub_dates(a):
  return (datetime.strptime('01-11-1998', date_format)-a).days   
def dat_strp(a):
    return datetime.strptime(a, date_format)
ratings['new_date']=ratings['rating_date'].apply(dat_strp)    
ratings['days_diff']=ratings['new_date'].apply(sub_dates) 
#conversion of no of days to years
ratings['years_diff']=round(ratings['days_diff']/365,2)
ratings.head() 

In [10]:
#dropping the below columns 
ratings.drop(['unix_timestamp','new_date'],axis=1,inplace=True)
ratings.head()

,user_id,movie_id,rating,rating_date,days_diff,years_diff
0,196,242,3,04-12-1997,332,0.91
1,186,302,3,04-04-1998,211,0.58
2,22,377,1,07-11-1997,359,0.98
3,244,51,2,27-11-1997,339,0.93
4,166,346,1,02-02-1998,272,0.75


In [ ]:
ratings['years_diff'].describe()

In [ ]:
#for these values select m=0.425 and n=0.25, this idea is based on max and min values of years difference to get weighted difference
m,n=0.425,0.25
ratings['weighted_diff']=m*ratings['years_diff']+n
ratings.head()

In [12]:
#Now I am gonna create new ratings which are time based/temporal
ratings['final_ratings']=round(ratings['rating']/ratings['weighted_diff'],2)
ratings.head()

,user_id,movie_id,rating,rating_date,days_diff,years_diff,weighted_diff,final_ratings
0,196,242,3,04-12-1997,332,0.91,0.63675,4.71
1,186,302,3,04-04-1998,211,0.58,0.49650,6.04
2,22,377,1,07-11-1997,359,0.98,0.66650,1.50
3,244,51,2,27-11-1997,339,0.93,0.64525,3.10
4,166,346,1,02-02-1998,272,0.75,0.56875,1.76


In [ ]:
ratings['final_ratings'].describe()

In [13]:
n_users = ratings.user_id.unique().shape[0]
n_items = ratings.movie_id.unique().shape[0]
data_matrix = np.zeros((n_users, n_items))

In [14]:
#basically here I am filling the respective user ids(line[1]) and movies(line[2]) with the value final ratings(line[8])
for line in ratings.itertuples():
    data_matrix[line[1]-1,line[2]-1] = line[8]
data_matrix    

array([[6.93, 4.31, 5.96, ..., 0.  , 0.  , 0.  ],
       [7.42, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [6.93, 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
       [0.  , 9.35, 0.  , ..., 0.  , 0.  , 0.  ]])

# **A Self Created Function For Matrix Factorization**

In [26]:
class MF():

    # Initializing the user-movie rating matrix, no. of latent features, alpha and beta.
    def __init__(self, R, K, alpha, beta, iterations,optimiz_algo):
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        #The below initializations have been done for Adam Optimization and RMS Prop algos
        self.m_dP,self.m_dQ,self.v_dP,self.v_dQ = 0,0,0,0
        self.m_db1,self.m_db2,self.v_db1,self.v_db2 = 0,0,0,0
        self.beta1 = 0.9
        self.beta2 = 0.99
        self.epsilon = 1e-8
        self.eta = 0.01
        self.optimiz_algo=optimiz_algo

    # Initializing user-feature and movie-feature matrix 
    def train(self):
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # List of training samples
        self.samples = [
        (i, j, self.R[i, j])
        for i in range(self.num_users)
        for j in range(self.num_items)
        if self.R[i, j] > 0
        ]

        # Stochastic gradient descent for given number of iterations
        training_process = []
        for i in range(self.iterations):
          
          t=i+1 #for use in Adam's optimization
          np.random.shuffle(self.samples)
          if(self.optimiz_algo==3):
            self.Adam(t)
          if(self.optimiz_algo==1):
            self.sgd()
          elif(self.optimiz_algo==2):
            self.rms()      
          mse = self.mse()
          training_process.append((i, mse))
          #to check outcome after every 20 iterations
          if (i+1) % 20 == 0:
            print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    # Computing total mean squared error
    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
        #print('mP=',self.P[i, :],'mQ=',self.Q[j, :],'b1=',self.b_u[i],'b2=',self.b_i[j]) 
    def rms(self):
        for i, j, r in self.samples:  
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            dP= -(e * self.Q[j, :] - self.beta * self.P[i,:])
            dQ= -(e * self.P[i, :] - self.beta * self.Q[j,:])
            db1= -(e - self.beta * self.b_u[i])
            db2= -(e - self.beta * self.b_i[j])
            ## rms beta 2
            # *** weights *** #
            self.v_dP = self.beta2*self.v_dP + (1-self.beta2)*(dP**2)
            self.v_dQ = self.beta2*self.v_dQ + (1-self.beta2)*(dQ**2)
            # *** biases *** #
            self.v_db1 = self.beta2*self.v_db1 + (1-self.beta2)*(db1**2)
            self.v_db2 = self.beta2*self.v_db2 + (1-self.beta2)*(db2**2)
            # update weights and biases
            self.P[i, :] = self.P[i, :] - self.eta*(dP/(np.sqrt(self.v_dP)+self.epsilon))
            self.Q[j, :] = self.Q[j, :] - self.eta*(dQ/(np.sqrt(self.v_dQ)+self.epsilon)) 
            self.b_u[i] = self.b_u[i] - self.eta*(db1/(np.sqrt(self.v_db1)+self.epsilon))
            self.b_i[j] = self.b_i[j] - self.eta*(db2/(np.sqrt(self.v_db2)+self.epsilon))
            
    def Adam(self,t):
        for i, j, r in self.samples:  
            prediction = self.get_rating(i, j)
            e = (r - prediction)    
            dP= -(e * self.Q[j, :] - self.beta * self.P[i,:])
            dQ= -(e * self.P[i, :] - self.beta * self.Q[j,:])
            db1= -(e - self.beta * self.b_u[i])
            db2= -(e - self.beta * self.b_i[j])
            self.m_dP = self.beta1*self.m_dP + (1-self.beta1)*dP
            self.m_dQ = self.beta1*self.m_dQ + (1-self.beta1)*dQ
            # *** biases *** #
            self.m_db1 = self.beta1*self.m_db1 + (1-self.beta1)*db1
            self.m_db2 = self.beta1*self.m_db2 + (1-self.beta1)*db2  
            ## rms beta 2
            # *** weights *** #
            self.v_dP = self.beta2*self.v_dP + (1-self.beta2)*(dP**2)
            self.v_dQ = self.beta2*self.v_dQ + (1-self.beta2)*(dQ**2)
            # *** biases *** #
            self.v_db1 = self.beta2*self.v_db1 + (1-self.beta2)*(db1**2)
            self.v_db2 = self.beta2*self.v_db2 + (1-self.beta2)*(db2**2)

            # bias correction
            m_dP_corr = self.m_dP/(1-self.beta1**t)
            m_dQ_corr = self.m_dQ/(1-self.beta1**t)
            m_db1_corr = self.m_db1/(1-self.beta1**t)
            m_db2_corr = self.m_db2/(1-self.beta1**t)
            v_dP_corr = self.v_dP/(1-self.beta2**t)
            v_dQ_corr = self.v_dQ/(1-self.beta2**t)
            v_db1_corr = self.v_db1/(1-self.beta2**t)
            v_db2_corr = self.v_db2/(1-self.beta2**t)
            #print('mP=',m_dP_corr,'mQ=',m_dQ_corr,'vP=',v_dP_corr,'vQ=',v_dQ_corr)  
            # update weights and biases
            self.P[i, :] = self.P[i, :] - self.eta*(m_dP_corr/(np.sqrt(v_dP_corr)+self.epsilon))
            self.Q[j, :] = self.Q[j, :] - self.eta*(m_dQ_corr/(np.sqrt(v_dQ_corr)+self.epsilon)) 
            self.b_u[i] = self.b_u[i] - self.eta*(m_db1_corr/(np.sqrt(v_db1_corr)+self.epsilon))
            self.b_i[j] = self.b_i[j] - self.eta*(m_db2_corr/(np.sqrt(v_db2_corr)+self.epsilon))
            #print('for Iteration=',t)
        #print('mP=',self.P[i, :],'mQ=',self.Q[j, :],'b1=',self.b_u[i],'b2=',self.b_i[j])  
        #print('val_of_t=',t)         
    # Ratings for user i and movie j
    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Full user-movie rating matrix
    def full_matrix(self):
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

R – The user-movie rating matrix
<br>
K – Number of latent features
<br>
alpha – Learning rate for stochastic gradient descent
<br>
beta – Regularization parameter for bias
<br>
iterations – Number of iterations to perform stochastic gradient descent

**In optimiz_algo argument pass 1 for stochastic gradient descent optimization, 2 for rms prop and 3 for Adam , we will check each of these**

In [20]:
#prediction of ratings using Matrix Factorization using sgd optimization Algo
R= np.array(ratings.pivot(index = 'user_id', columns ='movie_id', values = 'final_ratings').fillna(0))
mf = MF(R, K=20, alpha=0.01, beta=0.005, iterations=100,optimiz_algo=1)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 332.1550
Iteration: 40 ; error = 304.1338
Iteration: 60 ; error = 295.1879
Iteration: 80 ; error = 290.9069
Iteration: 100 ; error = 288.3020

P x Q:
[[ 5.85425303  4.72359115  6.57067244 ...  4.24006339  5.04275109
   4.7248933 ]
 [ 7.07216722  5.74623694  5.50216316 ...  5.64885351  6.43095164
   6.19339998]
 [ 5.88484623  4.62089542  3.34415432 ...  4.96156998  5.42685738
   5.2878281 ]
 ...
 [ 6.7385807   5.33042137  7.35364109 ...  4.69131941  5.34923002
   5.10220769]
 [ 7.91653043 10.33343282 11.62011094 ...  7.90288867  7.93364397
   7.87679205]
 [ 5.21625639  8.89849227  6.66468945 ...  5.16646079  4.99157564
   4.97139743]]



In [22]:
#After sgd
res=mf.full_matrix()
count=[]
for i in range(n_users):
    c=0
    #for each user we are getting his top 15 rated movies
    k=np.argsort(res[i])[::-1][:15]
    #Now we are checking how many of the highest rated movies the user has already watched
    for j in k:
      if data_matrix[i][j]>=res[i][::-1][:15].min():
        c+=1
    count.append(c)  
print('capture rate=',count) 
print('The total number of top 15 recommended movies which had been already watched by users is')       
print(sum(count))

capture rate= [3, 0, 1, 0, 0, 6, 2, 0, 0, 1, 2, 0, 3, 0, 0, 6, 0, 9, 0, 0, 0, 0, 4, 2, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 5, 1, 0, 0, 2, 0, 2, 0, 0, 1, 0, 0, 0, 1, 0, 0, 6, 1, 0, 3, 0, 6, 1, 0, 0, 0, 2, 4, 0, 0, 1, 3, 0, 3, 0, 0, 0, 0, 0, 2, 3, 0, 5, 0, 2, 0, 0, 1, 0, 4, 0, 4, 3, 0, 0, 0, 0, 4, 0, 3, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 5, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 2, 0, 0, 0, 0, 0, 4, 3, 0, 3, 0, 0, 0, 0, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 7, 2, 0, 0, 0, 0, 5, 0, 0, 0, 0, 1, 1, 3, 0, 0, 8, 1, 0, 0, 3, 0, 4, 6, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 1, 8, 0, 5, 0, 0, 1, 0, 0, 0, 0, 0, 0, 10, 4, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 0, 3, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 5, 7, 3, 0, 12, 0, 0, 0, 2, 13, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 1, 0, 0, 5, 3, 3, 10, 4, 0, 0, 5, 3, 4, 0, 3, 0, 11, 0, 1, 0, 0, 4, 0, 0, 6, 1, 2, 1, 0, 0, 0, 2, 0, 4, 2, 0, 0, 0, 1, 0, 5,

In [27]:
#prediction of ratings using Matrix Factorization using rms prop optimization Algo
R= np.array(ratings.pivot(index = 'user_id', columns ='movie_id', values = 'final_ratings').fillna(0))
mf = MF(R, K=20, alpha=0.01, beta=0.005, iterations=100,optimiz_algo=2)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 309.5850
Iteration: 40 ; error = 296.0973
Iteration: 60 ; error = 290.8980
Iteration: 80 ; error = 288.2814
Iteration: 100 ; error = 286.3889

P x Q:
[[ 5.53434395  4.43766373  7.16620716 ...  5.05713913  5.43460486
   4.74604279]
 [ 8.16441579  7.37529455  4.05407087 ...  6.68859905  6.88173089
   6.40705734]
 [ 6.89398565  5.51958212  3.70694877 ...  4.47734699  5.01219576
   4.74750081]
 ...
 [ 6.35202686  5.25491516  5.18518644 ...  5.87214315  5.52648234
   5.44309992]
 [ 9.64487516  7.16507709  4.25082935 ...  7.31804007  7.44562414
   7.48418854]
 [ 6.67894047  7.82820713 -0.03296048 ...  4.69902456  5.77422617
   5.75397514]]



In [28]:
#After rmsprop
res1=mf.full_matrix()
count1=[]
for i in range(n_users):
    c=0
    #for each user we are getting his top 15 rated movies
    k=np.argsort(res1[i])[::-1][:15]
    #Now we are checking how many of the highest rated movies the user has already watched
    for j in k:
      if data_matrix[i][j]>=res1[i][::-1][:15].min():
        c+=1
    count1.append(c)  
print('capture rate=',count1) 
print('The total number of top 15 recommended movies which had been already watched by users is')       
print(sum(count1))

capture rate= [3, 0, 0, 0, 0, 5, 1, 0, 0, 4, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 5, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 3, 1, 0, 4, 0, 3, 0, 0, 3, 0, 1, 0, 5, 1, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 5, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 1, 0, 0, 0, 0, 0, 5, 1, 0, 1, 0, 0, 0, 3, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 7, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 6, 0, 1, 8, 0, 0, 0, 0, 0, 3, 0, 0, 4, 0, 0, 1, 2, 1, 2, 0, 0, 0, 0, 1, 6, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 4, 4, 0, 0, 3, 0, 0, 0, 0, 9, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 5, 0, 0, 1, 2, 5, 4, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 3, 0, 0, 0, 3, 1, 1

In [29]:
#prediction of ratings using Matrix Factorization using Adam optimization Algo
R= np.array(ratings.pivot(index = 'user_id', columns ='movie_id', values = 'final_ratings').fillna(0))
mf = MF(R, K=20, alpha=0.01, beta=0.005, iterations=160,optimiz_algo=3)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())
print()

Iteration: 20 ; error = 503.4257
Iteration: 40 ; error = 478.8507
Iteration: 60 ; error = 446.0526
Iteration: 80 ; error = 411.0205
Iteration: 100 ; error = 383.3088
Iteration: 120 ; error = 363.7716
Iteration: 140 ; error = 350.1064
Iteration: 160 ; error = 340.2003

P x Q:
[[5.5727918  4.0080037  4.64918426 ... 4.39944441 5.07219784 5.03275912]
 [7.09342339 5.06114304 5.21529991 ... 5.19292252 6.35328728 5.92715057]
 [8.40235514 5.16139183 5.59160882 ... 5.02393914 5.50108328 5.56650589]
 ...
 [6.57410168 5.2719803  4.84561112 ... 5.75099558 6.0828026  5.90069269]
 [9.24729062 7.72292778 5.92061478 ... 6.7406584  7.58006459 6.96783783]
 [6.49620084 6.40128403 6.25369474 ... 5.33290406 5.9906621  5.45172374]]



In [30]:
#After Adam
res2=mf.full_matrix()
count2=[]
for i in range(n_users):
    c=0
    #for each user we are getting his top 15 rated movies
    k=np.argsort(res2[i])[::-1][:15]
    #Now we are checking how many of the highest rated movies the user has already watched
    for j in k:
      if data_matrix[i][j]>=res2[i][::-1][:15].min():
        c+=1
    count2.append(c)  
print('capture rate=',count2) 
print('The total number of top 15 recommended movies which had been already watched by users is')       
print(sum(count2))

capture rate= [2, 2, 2, 2, 1, 11, 10, 4, 2, 7, 2, 0, 7, 2, 2, 7, 2, 8, 0, 0, 2, 2, 6, 3, 4, 3, 2, 1, 2, 2, 2, 0, 3, 1, 0, 0, 0, 0, 2, 0, 3, 7, 7, 7, 3, 2, 2, 7, 3, 0, 6, 0, 2, 4, 0, 9, 5, 6, 4, 4, 1, 7, 3, 4, 2, 0, 0, 0, 2, 1, 3, 5, 8, 3, 0, 3, 4, 1, 2, 2, 2, 3, 3, 2, 12, 0, 7, 3, 2, 14, 6, 5, 0, 9, 8, 3, 3, 0, 4, 0, 0, 5, 0, 5, 5, 5, 2, 4, 6, 9, 4, 6, 1, 2, 3, 8, 1, 3, 6, 4, 6, 2, 3, 2, 5, 3, 0, 2, 2, 12, 4, 3, 4, 3, 1, 5, 3, 2, 1, 2, 4, 2, 5, 8, 8, 2, 3, 3, 5, 2, 5, 4, 0, 3, 0, 4, 1, 3, 2, 4, 4, 0, 6, 6, 3, 3, 2, 2, 4, 2, 3, 0, 1, 2, 3, 1, 10, 10, 6, 1, 8, 4, 1, 9, 4, 1, 3, 1, 6, 3, 3, 1, 3, 8, 1, 2, 4, 10, 0, 6, 14, 0, 1, 3, 0, 7, 7, 2, 2, 10, 1, 0, 3, 4, 3, 5, 0, 1, 0, 2, 3, 8, 5, 4, 1, 2, 1, 3, 4, 5, 2, 4, 3, 12, 2, 5, 4, 1, 1, 3, 2, 1, 4, 3, 2, 2, 4, 3, 4, 1, 1, 3, 4, 10, 4, 1, 3, 4, 0, 2, 0, 5, 5, 3, 1, 4, 5, 7, 8, 1, 12, 2, 5, 1, 3, 12, 1, 6, 7, 2, 0, 0, 2, 4, 3, 6, 3, 3, 0, 10, 4, 4, 12, 5, 2, 7, 5, 9, 8, 1, 11, 0, 12, 4, 6, 2, 5, 7, 2, 1, 8, 3, 9, 9, 3, 4, 2, 6, 2, 5, 6, 4, 3

In [31]:
#check prediction for the top rated movies by each of the optimizer models for user 0 for the movies he has watched
#sgd optimizer
np.argsort(res[0])[::-1][:15]

array([220, 170, 515, 380, 284, 136, 529, 962, 511, 344, 420, 708, 275,
       518, 640])

In [33]:
print('The true rating for movie id 170 by user 0 is=',data_matrix[0][170])
print('The predicted rating for movie id 170 by user 0 is=',res[0][170])

The true rating for movie id 170 by user 0 is= 9.58
The predicted rating for movie id 170 by user 0 is= 8.577174713123407


In [37]:
#rms optimizer
np.argsort(res1[0])[::-1][:15]

array([ 895,  170, 1061, 1072, 1066,  610, 1135,  651,  463,  908,    5,
        220, 1141,  512,  864])

In [43]:
print('The true rating for movie id 5 by user 0 is=',data_matrix[0][5])
print('The predicted rating for movie id 5 by user 0 is=',res1[0][5])

The true rating for movie id 5 by user 0 is= 9.06
The predicted rating for movie id 5 by user 0 is= 8.73903576949499


In [44]:
#Adam optimizer
np.argsort(res2[0])[::-1][:15]

array([168, 407, 646, 511, 656, 655, 512, 165, 473, 610, 515, 693, 301,
       483, 854])

In [50]:
print('The true rating for movie id 168 by user 0 is=',data_matrix[0][168])
print('The predicted rating for movie id 168 by user 0 is=',res2[0][168])

The true rating for movie id 168 by user 0 is= 7.45
The predicted rating for movie id 168 by user 0 is= 8.846200068782734
